## Import modules and create spark session

In [1]:
#import modules
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover

#create Spark session
appName = "Sentiment Analysis in Spark"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

## Read data file into Spark dataFrame

In [2]:
#read csv file into dataFrame with automatically inferred schema
tweets_csv = spark.read.csv('dataset/tweets.csv', inferSchema=True, header=True)
tweets_csv.show(truncate=False, n=3)

+------+---------+---------------+---------------------------------+
|ItemID|Sentiment|SentimentSource|SentimentText                    |
+------+---------+---------------+---------------------------------+
|1038  |1        |Sentiment140   |that film is fantastic #brilliant|
|1804  |1        |Sentiment140   |this music is really bad #myband |
|1693  |0        |Sentiment140   |winter is terrible #thumbs-down  |
+------+---------+---------------+---------------------------------+
only showing top 3 rows



## Select the related data

In [3]:
#select only "SentimentText" and "Sentiment" column, 
#and cast "Sentiment" column data into integer
data = tweets_csv.select("SentimentText", col("Sentiment").cast("Int").alias("label"))
data.show(truncate = False,n=5)

+---------------------------------+-----+
|SentimentText                    |label|
+---------------------------------+-----+
|that film is fantastic #brilliant|1    |
|this music is really bad #myband |1    |
|winter is terrible #thumbs-down  |0    |
|this game is awful #nightmare    |0    |
|I love jam #loveit               |1    |
+---------------------------------+-----+
only showing top 5 rows



## Divide data into training and testing data

In [4]:
#divide data, 70% for training, 30% for testing
dividedData = data.randomSplit([0.7, 0.3]) 
trainingData = dividedData[0] #index 0 = data training
testingData = dividedData[1] #index 1 = data testing
train_rows = trainingData.count()
test_rows = testingData.count()
print ("Training data rows:", train_rows, "; Testing data rows:", test_rows)

Training data rows: 1325 ; Testing data rows: 607


## Prepare training data

Separate "SentimentText" into individual words using tokenizer

In [5]:
tokenizer = Tokenizer(inputCol="SentimentText", outputCol="SentimentWords")
tokenizedTrain = tokenizer.transform(trainingData)
tokenizedTrain.show(truncate=False, n=5)

+----------------------------------+-----+----------------------------------------+
|SentimentText                     |label|SentimentWords                          |
+----------------------------------+-----+----------------------------------------+
|I adore cheese #brilliant         |1    |[i, adore, cheese, #brilliant]          |
|I adore cheese #thumbs-up         |1    |[i, adore, cheese, #thumbs-up]          |
|I adore cheese #toptastic         |1    |[i, adore, cheese, #toptastic]          |
|I adore classical music #brilliant|1    |[i, adore, classical, music, #brilliant]|
|I adore classical music #favorite |1    |[i, adore, classical, music, #favorite] |
+----------------------------------+-----+----------------------------------------+
only showing top 5 rows



Removing stop words (unimportant words to be features)

In [7]:
swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(), 
                       outputCol="MeaningfulWords")
SwRemovedTrain = swr.transform(tokenizedTrain)
SwRemovedTrain.show(truncate=False, n=5)

+----------------------------------+-----+----------------------------------------+-------------------------------------+
|SentimentText                     |label|SentimentWords                          |MeaningfulWords                      |
+----------------------------------+-----+----------------------------------------+-------------------------------------+
|I adore cheese #brilliant         |1    |[i, adore, cheese, #brilliant]          |[adore, cheese, #brilliant]          |
|I adore cheese #thumbs-up         |1    |[i, adore, cheese, #thumbs-up]          |[adore, cheese, #thumbs-up]          |
|I adore cheese #toptastic         |1    |[i, adore, cheese, #toptastic]          |[adore, cheese, #toptastic]          |
|I adore classical music #brilliant|1    |[i, adore, classical, music, #brilliant]|[adore, classical, music, #brilliant]|
|I adore classical music #favorite |1    |[i, adore, classical, music, #favorite] |[adore, classical, music, #favorite] |
+-----------------------

Converting words feature into numerical feature. In Spark 2.2.1,it is implemented in HashingTF funtion using Austin Appleby's MurmurHash 3 algorithm

In [8]:
hashTF = HashingTF(inputCol=swr.getOutputCol(), outputCol="features")
numericTrainData = hashTF.transform(SwRemovedTrain).select(
    'label', 'MeaningfulWords', 'features')
numericTrainData.show(truncate=False, n=3)

+-----+---------------------------+------------------------------------------+
|label|MeaningfulWords            |features                                  |
+-----+---------------------------+------------------------------------------+
|1    |[adore, cheese, #brilliant]|(262144,[61111,65702,69876],[1.0,1.0,1.0])|
|1    |[adore, cheese, #thumbs-up]|(262144,[3984,65702,69876],[1.0,1.0,1.0]) |
|1    |[adore, cheese, #toptastic]|(262144,[65702,69876,82232],[1.0,1.0,1.0])|
+-----+---------------------------+------------------------------------------+
only showing top 3 rows



## Train our classifier model using training data

In [9]:
lr = LogisticRegression(labelCol="label", featuresCol="features", 
                        maxIter=10, regParam=0.01)
model = lr.fit(numericTrainData)
print ("Training is done!")

Training is done!


## Prepare testing data

In [11]:
tokenizedTest = tokenizer.transform(testingData)
SwRemovedTest = swr.transform(tokenizedTest)
numericTest = hashTF.transform(SwRemovedTest).select(
    'Label', 'MeaningfulWords', 'features')
numericTest.show(truncate=False, n=2)


+-----+--------------------------+-------------------------------------------+
|Label|MeaningfulWords           |features                                   |
+-----+--------------------------+-------------------------------------------+
|1    |[adore, cheese, #bestever]|(262144,[65702,69876,108823],[1.0,1.0,1.0])|
|1    |[adore, cheese, #favorite]|(262144,[65702,69876,156543],[1.0,1.0,1.0])|
+-----+--------------------------+-------------------------------------------+
only showing top 2 rows



## Predict testing data and calculate the accuracy model

In [12]:
prediction = model.transform(numericTest)
predictionFinal = prediction.select(
    "MeaningfulWords", "prediction", "Label")
predictionFinal.show(n=4, truncate = False)
correctPrediction = predictionFinal.filter(
    predictionFinal['prediction'] == predictionFinal['Label']).count()
totalData = predictionFinal.count()
print("correct prediction:", correctPrediction, ", total data:", totalData, 
      ", accuracy:", correctPrediction/totalData)

+------------------------------------+----------+-----+
|MeaningfulWords                     |prediction|Label|
+------------------------------------+----------+-----+
|[adore, cheese, #bestever]          |1.0       |1    |
|[adore, cheese, #favorite]          |1.0       |1    |
|[adore, cheese, #loveit]            |1.0       |1    |
|[adore, classical, music, #bestever]|1.0       |1    |
+------------------------------------+----------+-----+
only showing top 4 rows

correct prediction: 597 , total data: 607 , accuracy: 0.9835255354200988
